In [ ]:
###########################################################################
# Package Imports and Database Connection Settings
###########################################################################

# Package imports
import psycopg2
import pandas.io.sql as pdsql

# Set database connection parameters
params = {
  'dbname': 'clicc',
  'user': 'clicc',
  'password': 'clicc',
  'host': '192.168.99.100',
  'port': 5432
}

# Attempt to connect to a PostGIS database running in docker container
try: 
    conn = psycopg2.connect(**params)
    print('Connected to PostGIS Database')
except: 
    print('Failed to Connect to PostGIS Database')

In [7]:
###########################################################################
# Task #1
###########################################################################

# Build custom SQL query to compute landuse fractions
queryLF = """
    SELECT huc, 
    (ST_ValueCount(ST_Union(ST_Clip(rast,geom)))).*
       INTO cat_counts 
       FROM landuse, catchments 
       GROUP BY huc; 
    SELECT huc, value, count, 
    (SUM(count) OVER (PARTITION BY huc)) AS sum 
        INTO cat_counts_sum 
        FROM cat_counts;
    SELECT huc, value, count, 
    (@ST_ScaleX(rast) * @ST_ScaleY(rast) * (CAST(count AS float))) AS area, 
    (CAST(count AS float) / CAST(sum AS float)) AS fraction
        FROM landuse, cat_counts_sum
        ORDER BY huc, count;
"""

# Execute query and write to pandas data frame
LF = pdsql.read_sql(queryLF, conn)

# Print data frame contents
print(LF)

         huc  value    count          area  fraction
0   17090003     31    25179  2.266296e+07  0.004757
1   17090003     95    36819  3.313982e+07  0.006956
2   17090003     24    39965  3.597145e+07  0.007551
3   17090003     41    41941  3.775000e+07  0.007924
4   17090003     11    52508  4.726108e+07  0.009921
5   17090003     23    91297  8.217405e+07  0.017249
6   17090003     90   154410  1.389804e+08  0.029174
7   17090003     22   190494  1.714587e+08  0.035991
8   17090003     21   232764  2.095048e+08  0.043977
9   17090003     71   241475  2.173454e+08  0.045623
10  17090003     43   334741  3.012916e+08  0.063245
11  17090003     82   598431  5.386321e+08  0.113065
12  17090003     52   632269  5.690888e+08  0.119458
13  17090003     42  1137751  1.024060e+09  0.214962
14  17090003     81  1482763  1.334596e+09  0.280147
15  17090005     24     2451  2.206081e+06  0.001095
16  17090005     95     3040  2.736225e+06  0.001358
17  17090005     12     3406  3.065652e+06  0.

In [6]:
###########################################################################
# Task #2
###########################################################################

# Build custom SQL query to compute mean reach depth
queryRD = """
    SELECT huc, 
    AVG(pdepth) AS mean_depth 
        FROM reaches
        GROUP BY huc
        ORDER BY mean_depth;
"""

# Execute query and write to pandas data frame
RD = pdsql.read_sql(queryRD, conn)

# Print data frame contents
print(RD)

        huc  mean_depth
0  17090008    0.286947
1  17090009    0.449966
2  17090006    1.037389
3  17090003    1.364943
4  17090007    2.071696
5  17090005    3.095397


In [5]:
###########################################################################
# Task #3
###########################################################################

# Build custom SQL query to compute the distribution of landuses per
# soil classification zone
querySG = """
    SELECT huc, muid, 
    ST_Intersection(catchments.geom,soils.geom) AS int_geom
        INTO soils_int
        FROM catchments, soils
        GROUP BY huc, muid, catchments.geom, soils.geom;
    SELECT huc, muid, 
    (ST_ValueCount(ST_Union(ST_Clip(rast,int_geom)))).* AS counts
        INTO soils_counts
        FROM landuse, soils_int
        GROUP BY huc, muid;
    SELECT huc, muid, value, count, 
    (SUM(count) OVER (PARTITION BY muid)) AS sum 
        INTO soils_counts_sum 
        FROM soils_counts;
    SELECT huc, muid, value, count, 
    (@ST_ScaleX(rast) * @ST_ScaleY(rast) * (CAST(count AS float))) AS area, 
    (CAST(count AS float) / CAST(sum AS float)) AS fraction
        FROM landuse, soils_counts_sum
        ORDER BY huc, muid, count;
"""

# Execute query and write to pandas data frame
SG = pdsql.read_sql(querySG, conn)

# Print data frame contents
print(SG)

          huc   muid  value   count             area  fraction
0    17090003  OR064     21       4      3600.295717  0.000002
1    17090003  OR064     22       7      6300.517505  0.000004
2    17090003  OR064     31      17     15301.256799  0.000009
3    17090003  OR064     41      62     55804.583618  0.000034
4    17090003  OR064     43      98     88207.245074  0.000053
5    17090003  OR064     71    4604   4143940.370626  0.002503
6    17090003  OR064     52   14282  12854855.858661  0.007765
7    17090003  OR064     42   47398  42661704.102283  0.025771
8    17090003  OR068     23       7      6300.517505  0.000033
9    17090003  OR068     82      68     61205.027194  0.000320
10   17090003  OR068     11      90     81006.653640  0.000424
11   17090003  OR068     95     107     96307.910438  0.000504
12   17090003  OR068     81     208    187215.377300  0.000979
13   17090003  OR068     22     232    208817.151604  0.001092
14   17090003  OR068     31    1410   1269104.240352  0